In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
def analyze_stock(ticker_symbol, period='2y'):
    # Download stock data
    stock = yf.Ticker(ticker_symbol)
    hist = stock.history(period=period)
    
    # Calculate key metrics
    hist['SMA_50'] = hist['Close'].rolling(window=50).mean()
    hist['SMA_200'] = hist['Close'].rolling(window=200).mean()
    hist['Daily_Return'] = hist['Close'].pct_change()
    hist['Volatility'] = hist['Daily_Return'].rolling(window=20).std() * np.sqrt(252) * 100
    
    # Create the visualization
    fig = make_subplots(rows=3, cols=1, 
                        subplot_titles=('Price and Moving Averages', 'Volume', 'Volatility'),
                        vertical_spacing=0.1,
                        row_heights=[0.5, 0.25, 0.25])

    # Price and Moving Averages
    fig.add_trace(
        go.Scatter(x=hist.index, y=hist['Close'], name='Close Price', line=dict(color='blue')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=hist.index, y=hist['SMA_50'], name='50-day SMA', line=dict(color='orange')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=hist.index, y=hist['SMA_200'], name='200-day SMA', line=dict(color='red')),
        row=1, col=1
    )

    # Volume
    fig.add_trace(
        go.Bar(x=hist.index, y=hist['Volume'], name='Volume', marker_color='gray'),
        row=2, col=1
    )

    # Volatility
    fig.add_trace(
        go.Scatter(x=hist.index, y=hist['Volatility'], name='20-day Volatility', 
                  line=dict(color='purple')),
        row=3, col=1
    )

    fig.update_layout(
        title=f'{ticker_symbol} Stock Analysis',
        height=1000,
        showlegend=True
    )

    return fig, hist, stock.info

In [ ]:
fig, hist_data, company_info = analyze_stock('RBLX')

fig.show()

In [ ]:
# Calculate key statistics
current_price = hist_data['Close'][-1]
peak_price = hist_data['Close'].max()
drawdown = ((peak_price - current_price) / peak_price) * 100
volatility = hist_data['Daily_Return'].std() * np.sqrt(252) * 100
total_return = ((hist_data['Close'][-1] / hist_data['Close'][0]) - 1) * 100

stats = pd.DataFrame({
    'Metric': ['Current Price', 'Peak Price', 'Drawdown from Peak', 'Annual Volatility', 'Total Return'],
    'Value': [
        f"${current_price:.2f}",
        f"${peak_price:.2f}",
        f"{drawdown:.1f}%",
        f"{volatility:.1f}%",
        f"{total_return:.1f}%"
    ]
})

display(stats)

In [ ]:
company_metrics = pd.DataFrame({
    'Metric': ['Market Cap', 'Forward P/E', 'Price to Book', 'Enterprise Value', 'Revenue Growth'],
    'Value': [
        f"${company_info.get('marketCap', 'N/A'):,.0f}" if company_info.get('marketCap') else 'N/A',
        f"{company_info.get('forwardPE', 'N/A'):.2f}" if company_info.get('forwardPE') else 'N/A',
        f"{company_info.get('priceToBook', 'N/A'):.2f}" if company_info.get('priceToBook') else 'N/A',
        f"${company_info.get('enterpriseValue', 'N/A'):,.0f}" if company_info.get('enterpriseValue') else 'N/A',
        f"{company_info.get('revenueGrowth', 'N/A'):.1%}" if company_info.get('revenueGrowth') else 'N/A'
    ]
})

display(company_metrics)